In [12]:
import pandas as pd
import pandas_datareader.data as pdr
from enum import Enum
from datetime import datetime, timedelta
import requests as re
import json
import urllib as ur

In [2]:
def get_price_data(tic_list,method,interval = 1, freq = 'minutes',start_date = datetime.now()-timedelta(days =256),end_date=datetime.now()):
    

    error = []
    price = pd.DataFrame()
    if method == "robinhood":
        robinhood = get_robinhood()

        for i in tic_list:
            trial = 0
            while trial <3:
                try:
                    temp = robinhood.get_historical(i,interval="10minute",span = "week")
                    #temp.index = temp.begins_at
                    temp = temp.drop(["session","interpolated"],axis=1)
                    for j in temp.columns[1:]:
                        temp[j] = temp[j].astype(float)
                   # index= pd.MultiIndex.from_product([[i],temp.index])
                    temp["Ticker"] = np.repeat(i,len(temp))
                    price = price.append(temp)

                    print "Finished", i
                    #time.sleep(5)
                    trial=3

                except:
                    print "error occorded in getting yahool historicals for ", i
                    trial +=1
                    time.sleep(10)
                    if trial == 3:
                        error.append([i,'get_yahoo_historicals'])
        # get rid of the multiindex
       # price.to_csv("file/temp.csv")
       # price = pd.read_csv("file/temp.csv")

        price = price.reset_index()
        price["Close"] = price["close_price"]
        price = price.rename(columns={'begins_at':"TimeStamp","high_price":"High","low_price":"Low","open_price":"Open","volume":"Volume"})
        price["Return"]= (price.Close.diff(1)/price.Close)


    elif method == "ondemand":


        #freq = minutes, daily
        save_file_name = "Trade_suggestion_ondemand"
        for i in tic_list:
            trial = 0
            while trial <3:
                try:
                    temp = get_ondemand_data(i, interval = interval , freq = freq,start_date = start_date, end_date = end_date)

                    #index= pd.MultiIndex.from_product([[i],temp.index])
                    #temp=pd.DataFrame(data=temp.values,index=index,columns=temp.columns)

                    price = price.append(temp)

                    print "Finished", i
                    #time.sleep(5)
                    trial=3

                except:
                    print "error occorded in getting yahool historicals for ", i
                    trial +=1
                    time.sleep(10)
                    if trial == 3:
                        error.append([i,'get_yahoo_historicals'])


        price = price.reset_index()
        price["Close"] = price["close"]
        price = price.rename(columns={'symbol':'Ticker','timestamp':"TimeStamp","high":"High","low":"Low","open":"Open","volume":"Volume"})
        price["Return"]=( price.Close.diff(1)/price.Close)

    elif method == "day":
        save_file_name = "Trade_suggestion_day"
        mongodb = mongo()
        for i in tic_list:
            trial = 0
            while trial <3:
                try:
                    temp = mongodb.query_database(i,start_date=start_date,end_date=end_date)
                    index= pd.MultiIndex.from_product([[i],temp.index])
                    temp=pd.DataFrame(data=temp.values,index=index,columns=temp.columns)
                    price = price.append(temp)
                    print "Finished", i
                    trial = 3
                except Exception as e:
                    try:
                        print (e)
                        print ("Switching from Mongodb to yahoo")
                        temp = da.DataReader(i,"yahoo",start_date ,end_date)
                        index= pd.MultiIndex.from_product([[i],temp.index])
                        temp=pd.DataFrame(data=temp.values,index=index,columns=temp.columns)
                        price = price.append(temp)

                        print "Finished", i
                        #time.sleep(5)
                        trial=3


                    except Exception as e:
                        print "error occorded in getting yahool historicals for ", i
                        trial =3
                        trial +=1
                        time.sleep(10)
                    if trial == 3:
                        error.append([i,'get_yahoo_historicals'])
        # get rid of the multiindex
    elif method == "realtimeday":
        save_file_name = "Trade_suggestion_day"
        mongodb = mongo()
        for i in tic_list:
            trial = 0
            while trial <3:
                try:
                    temp = pdr.get_data_yahoo(i,start_date ,end_date)
                    index= pd.MultiIndex.from_product([[i],temp.index])
                    temp=pd.DataFrame(data=temp.values,index=index,columns=temp.columns)
                    price = price.append(temp)
                    print "Finished", i
                    trial = 3
                except Exception as e:
                    try:
                        print (e)
                        print "Finished", i
                        #time.sleep(5)
                        trial=3
                        print e

                    except Exception as e:
                        print "error occorded in getting yahool historicals for ", i
                        trial =3
                        trial +=1
                        time.sleep(10)
                    if trial == 3:
                        error.append([i,'get_yahoo_historicals'])
        price = price.reset_index()


        price["Close"] = price["Adj Close"]
        price = price.rename(columns={'level_0':'Ticker','level_1':"TimeStamp"})
        price["Return"]= (price.Close.diff(1)/price.Close)
    #price = price.dropna()
    return price

In [37]:

# tic_list,method,interval = 1, freq = 'minutes',start_date = datetime.now()-timedelta(days =256),end_date=datetime.now()

# 1min, 5min, 15min, 30min, 60min

# https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=5min&apikey=demo&datatype=csv

def get_price(tic_list,**kwargs):
    
    api = "R9KJYWUBMIDAXNW7"
    
    if len(kwargs) == 0 :
        start_date = datetime.now()-timedelta(days =256)
        end_date=datetime.now()
        data_interval="_day"
        data_size = "compact"
    
    data_interval = "_"+data_interval
    
    class interval(Enum):
        _1min = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=1min&outputsize={}&apikey={}&datatype=csv".format(tic_list,data_size,api)
        _5min = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=5min&outputsize={}&apikey={}&datatype=csv".format(tic_list,data_size,api)
        _15min = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=15min&outputsize={}&apikey={}&datatype=csv".format(tic_list,data_size,api)
        _30min = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=30min&outputsize={}&apikey={}&datatype=csv".format(tic_list,data_size,api)
        _60min = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=60min&outputsize={}&apikey={}&datatype=csv".format(tic_list,data_size,api)
        _week = " https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={}&apikey&outputsize={}=csv".format(tic_list,data_size,api)
        _month = " https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={}&apikey={}&outputsize={}&datatype=csv".format(tic_list,data_size,api)
        
    
    if data_interval == "_day":
        
        price = pdr.DataReader(tic_list,"quandl",start,end)
        price = pdr.DataReader("AAPL","quandl")
        price = price.reset_index()
        price = price.drop(["ExDividend","SplitRatio","Open","High","Low","Close","Volume"],axis=1)
        price = price.rename(columns={'AdjOpen':'Open','AdjHigh':"High","AdjLow":"Low","AdjVolume":"Volume","AdjClose":"Close"})
        price["Return"]= (price.Close.diff(1)/price.Close)
        
    else:
    
        price = pd.read_csv(interval[data_interval])
        price = pdr.DataReader("AAPL","quandl")
        price = price.reset_index()
        price = price.drop(["ExDividend","SplitRatio","Open","High","Low","Close","Volume"],axis=1)
        price = price.rename(columns={'AdjOpen':'Open','AdjHigh':"High","AdjLow":"Low","AdjVolume":"Volume","AdjClose":"Close"})
        price["Return"]= (price.Close.diff(1)/price.Close)
        

            
        

        
    
    

SyntaxError: invalid syntax (<ipython-input-37-35d9a03bdd9a>, line 39)

In [46]:
data_interval="_1min"
tic_list = "AAPL"
data_size = "compact"
api = "R9KJYWUBMIDAXNW7"

class interval(Enum):
        _1min = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=1min&outputsize={}&apikey={}&datatype=csv".format(tic_list,data_size,api)
        _5min = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=5min&outputsize={}&apikey={}&datatype=csv".format(tic_list,data_size,api)
        _15min = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=15min&outputsize={}&apikey={}&datatype=csv".format(tic_list,data_size,api)
        _30min = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=30min&outputsize={}&apikey={}&datatype=csv".format(tic_list,data_size,api)
        _60min = "https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={}&interval=60min&outputsize={}&apikey={}&datatype=csv".format(tic_list,data_size,api)
        _week = " https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol={}&apikey&outputsize={}=csv".format(tic_list,data_size,api)
        _month = " https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={}&apikey={}&outputsize={}&datatype=csv".format(tic_list,data_size,api)
        
price = pd.read_csv(interval[data_interval].value)
price = price.reset_index()
price = price.drop(["ExDividend","SplitRatio","Open","High","Low","Close","Volume"],axis=1)
price = price.rename(columns={'AdjOpen':'Open','AdjHigh':"High","AdjLow":"Low","AdjVolume":"Volume","AdjClose":"Close"})
price["Return"]= (price.Close.diff(1)/price.Close)        
    

ValueError: labels ['ExDividend' 'SplitRatio' 'Open' 'High' 'Low' 'Close' 'Volume'] not contained in axis

In [47]:
price

,index,timestamp,open,high,low,close,volume
0,0,2018-09-21 15:59:00,218.0250,218.3500,217.6400,217.6500,1657917
1,1,2018-09-21 15:58:00,217.8500,218.0400,217.7700,218.0100,488382
2,2,2018-09-21 15:57:00,217.8900,217.9400,217.7100,217.8550,330932
3,3,2018-09-21 15:56:00,217.8666,218.0900,217.8666,217.8900,395279
4,4,2018-09-21 15:55:00,217.8600,217.9000,217.6600,217.8600,443371
5,5,2018-09-21 15:54:00,217.8299,217.9425,217.8200,217.8700,294660
6,6,2018-09-21 15:53:00,217.5000,217.8600,217.5000,217.8250,321956
7,7,2018-09-21 15:52:00,217.5800,217.6350,217.4700,217.5200,195801
8,8,2018-09-21 15:51:00,217.5600,217.6650,217.4700,217.5900,339615
9,9,2018-09-21 15:50:00,217.4000,218.1000,217.3700,217.5900,931511
